In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [38]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import time

In [4]:
movies        = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/movies.csv')
links         = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/links.csv')
ratings       = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/ratings.csv')
tags          = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/tags.csv')

In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [7]:
movies.isnull().sum()

,0
movieId,0
title,0
genres,0


In [8]:
print(movies['genres'].nunique())
print(movies['genres'].unique())

951
['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Drama' 'Action|Thriller'
 'Drama|Horror|Thriller' 'Comedy|Horror|Roman

In [9]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [10]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [11]:
links.isnull().sum() # There is some missing values in links dataframe

,0
movieId,0
imdbId,0
tmdbId,8


In [12]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [13]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [14]:
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [15]:
ratings['userId'].nunique()

610

In [16]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [17]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [18]:
tags.isnull().sum()

,0
userId,0
movieId,0
tag,0
timestamp,0


In [19]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [20]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [21]:
merged_df = pd.merge(ratings, links, on='movieId')

In [22]:
merged_df

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,1,4.0,964982703,114709,862.0
1,5,1,4.0,847434962,114709,862.0
2,7,1,4.5,1106635946,114709,862.0
3,15,1,2.5,1510577970,114709,862.0
4,17,1,4.5,1305696483,114709,862.0
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,118745,30948.0
100832,610,160527,4.5,1479544998,66806,90351.0
100833,610,160836,3.0,1493844794,798722,70193.0
100834,610,163937,3.5,1493848789,1540011,351211.0


In [23]:
# Check for null values in the merged DataFrame
null_values = merged_df.isnull().sum()

# Display the count of null values for each column
print(null_values)

userId        0
movieId       0
rating        0
timestamp     0
imdbId        0
tmdbId       13
dtype: int64


In [24]:
merged_df = merged_df.dropna(subset=['tmdbId'])
merged_df

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,1,4.0,964982703,114709,862.0
1,5,1,4.0,847434962,114709,862.0
2,7,1,4.5,1106635946,114709,862.0
3,15,1,2.5,1510577970,114709,862.0
4,17,1,4.5,1305696483,114709,862.0
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,118745,30948.0
100832,610,160527,4.5,1479544998,66806,90351.0
100833,610,160836,3.0,1493844794,798722,70193.0
100834,610,163937,3.5,1493848789,1540011,351211.0


# Initial Model Setup and Baseline Performance

## Transoformer Model

In [36]:
# Encode userId, movieId, imdbId, and tmdbId as integers
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
imdb_encoder = LabelEncoder()
tmdb_encoder = LabelEncoder()

merged_df['userId'] = user_encoder.fit_transform(merged_df['userId'])
merged_df['movieId'] = movie_encoder.fit_transform(merged_df['movieId'])
merged_df['imdbId'] = imdb_encoder.fit_transform(merged_df['imdbId'])
merged_df['tmdbId'] = tmdb_encoder.fit_transform(merged_df['tmdbId'])

# Get the number of unique values for each feature
num_users = merged_df['userId'].nunique()
num_movies = merged_df['movieId'].nunique()
num_imdb = merged_df['imdbId'].nunique()
num_tmdb = merged_df['tmdbId'].nunique()

# Prepare input features and labels
X = merged_df[['userId', 'movieId', 'imdbId', 'tmdbId']]
y = merged_df['rating'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Transformer-based recommendation model
class TransformerRecommender(Model):
    def __init__(self, num_users, num_movies, num_imdb, num_tmdb, embed_dim, num_heads, ff_dim, num_transformer_blocks, rate=0.1):
        super(TransformerRecommender, self).__init__()

        # Embedding layers
        self.user_embedding = Embedding(num_users, embed_dim)
        self.movie_embedding = Embedding(num_movies, embed_dim)
        self.imdb_embedding = Embedding(num_imdb, embed_dim)
        self.tmdb_embedding = Embedding(num_tmdb, embed_dim)

        # Transformer layers
        self.transformer_blocks = [
            [
                MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim),
                LayerNormalization(epsilon=1e-6),
                Dense(ff_dim, activation="relu"),
                Dense(embed_dim),
                Dropout(rate)
            ]
            for _ in range(num_transformer_blocks)
        ]

        # Dense output layers
        self.global_pooling = GlobalAveragePooling1D()
        self.dropout = Dropout(rate)
        self.dense1 = Dense(ff_dim, activation='relu')
        self.dense2 = Dense(1, activation='linear')

    def call(self, inputs):
        user_id, movie_id, imdb_id, tmdb_id = inputs

        # Embedding lookup
        user_emb = self.user_embedding(user_id)
        movie_emb = self.movie_embedding(movie_id)
        imdb_emb = self.imdb_embedding(imdb_id)
        tmdb_emb = self.tmdb_embedding(tmdb_id)

        # Stack embeddings to create a sequence
        x = tf.stack([user_emb, movie_emb, imdb_emb, tmdb_emb], axis=1)

        # Pass through transformer blocks
        for mha, ln, ff1, ff2, dropout in self.transformer_blocks:
            attn_output = mha(x, x)
            x = ln(x + attn_output)
            ff_output = ff1(x)
            ff_output = ff2(ff_output)
            x = ln(x + dropout(ff_output))

        # Global pooling and dense layers
        x = self.global_pooling(x)
        x = self.dropout(x)
        x = self.dense1(x)
        output = self.dense2(x)

        return output

# Model hyperparameters
embed_dim = 32
num_heads = 4
ff_dim = 128
num_transformer_blocks = 2
dropout_rate = 0.1

# Instantiate and compile the model
model = TransformerRecommender(num_users=num_users, num_movies=num_movies, num_imdb=num_imdb, num_tmdb=num_tmdb,
                               embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim,
                               num_transformer_blocks=num_transformer_blocks, rate=dropout_rate)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model summary
model.build([(None,), (None,), (None,), (None,)])  # Fixes the build issue
model.summary()

# Prepare input data for training
train_user_input = X_train['userId'].values
train_movie_input = X_train['movieId'].values
train_imdb_input = X_train['imdbId'].values
train_tmdb_input = X_train['tmdbId'].values

test_user_input = X_test['userId'].values
test_movie_input = X_test['movieId'].values
test_imdb_input = X_test['imdbId'].values
test_tmdb_input = X_test['tmdbId'].values

# Start timer for training time
start_time = time.time()

# Train the model
history = model.fit([train_user_input, train_movie_input, train_imdb_input, train_tmdb_input], y_train,
                    epochs=10, batch_size=64,
                    validation_data=([test_user_input, test_movie_input, test_imdb_input, test_tmdb_input], y_test))



<ipython-input-36-65c211c6611f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['userId'] = user_encoder.fit_transform(merged_df['userId'])
<ipython-input-36-65c211c6611f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['movieId'] = movie_encoder.fit_transform(merged_df['movieId'])
<ipython-input-36-65c211c6611f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

Model: "transformer_recommender_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_17 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_18 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_19 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ multi_head_attention_8               │ ?                           │     0 (unbuilt) │
│ (MultiHeadAttention)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_8                │ ?                           │     0 (unbuilt) │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ multi_head_attention_9               │ ?                           │     0 (unbuilt) │
│ (MultiHeadAttention)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_9                │ ?                           │     0 (unbuilt) │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 52s 34ms/step - loss: 1.1512 - mean_absolute_error: 0.8106 - val_loss: 0.7629 - val_mean_absolute_error: 0.6756
Epoch 2/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 34s 27ms/step - loss: 0.6921 - mean_absolute_error: 0.6391 - val_loss: 0.7504 - val_mean_absolute_error: 0.6715
Epoch 3/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 34s 27ms/step - loss: 0.6212 - mean_absolute_error: 0.6008 - val_loss: 0.7480 - val_mean_absolute_error: 0.6638
Epoch 4/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - loss: 0.5826 - mean_absolute_error: 0.5791 - val_loss: 0.7561 - val_mean_absolute_error: 0.6619
Epoch 5/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 35s 27ms/step - loss: 0.5482 - mean_absolute_error: 0.5587 - val_loss: 0.7677 - val_mean_absolute_error: 0.6685
Epoch 6/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - loss: 0.5257 - mean_absolute_error: 0.5438 - val_loss: 0.7930 - val_mean_absolute_error: 0.6749
Epoch 7/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 35s 27ms/step - loss: 0.4960 - mea

In [39]:
# Predict on the test set
y_pred = model.predict([test_user_input, test_movie_input, test_imdb_input, test_tmdb_input]).flatten()

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

631/631 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step
Mean Squared Error (MSE): 0.8295250005106993
Root Mean Squared Error (RMSE): 0.9107826307691091
Mean Absolute Error (MAE): 0.689045895563294
